In [1]:
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도나 속도에 영향을 미치는 값)
MY_HIDDEN = 1024

In [2]:
raw = pd.DataFrame([['Flounder', '문치가자미'],
    ['Sole', '돌가자미'],
    ['BlackSeaBream', '감성돔'],
    ['Eel', '뱀장어'],
    ['Mackerel', '미유기'],
    ['Gudgeon', '뚝지'],
    ['Abalone', '전복'],
    ['GiantSeaBass', '대농갱이'],
    ['Catfish', '메기'],
    ['Loach', '동자개'],
    ['Sweetfish', '은어'],
    ['Bass', '쏘가리'],
    ['Carp', '붕어'],
    ['Turbot', '넙치'],
    ['Rockfish', '쥐노래미'],
    ['Cod', '대구'],
    ['Pollock', '명태'],
    ['GiantOctopus', '대문어'],
    ['FreshwaterCrab', '참게'],
    ['Salmon', '연어'],
    ['SeaCucumber', '해삼'],
    ['Cuttlefish', '갑오징어'],
    ['Mudfish', '미꾸라지'],
    ['Loach', '미꾸리'],
    ['Prawn', '대하'],
    ['Icefish', '빙어'],
    ['Crab', '꽃게'],
    ['SpottedSeaBass', '점농어'],
    ['YellowPuffer', '황복'],
    ['RockBream', '조피볼락'],
    ['WebfootOctopus', '주꾸미'],
    ['NortheastFreshwaterCrab', '동남참게'],
    ['Bitterling', '버들치'],
    ['Shrimp', '보리새우'],
    ['Rockfish', '볼락'],
    ['Carp', '잉어'],
    ['Gudgeon', '꼼치'],
    ['Dab', '돌돔'],
    ['Mackerel', '전어'],
    ['BlackRockfish', '왕우럭'],
    ['GiantSeaBream', '말쥐치'],
    ['SoftshelledTurtle', '참돔'],
    ['RedShrimp', '물렁가시붉은새우'],
    ['Gudgeon', '개볼락'],
    ['RedSeaCucumber', '홍해삼'],
    ['BlackSeaCucumber', '흑해삼'],
    ['FlowerCrab', '톱날꽃게'],
    ['Gudgeon', '쥐치'],
    ['YellowCroaker', '황어'],
    ['SeaBream', '참조기'],
    ['Bream', '부세'],
    ['Octopus', '낙지'],
    ['GiantSeaBass', '능성어'],
    ['GiantSeaBream', '민어'],
    ['Flatfish', '박대'],
    ['Grouper', '자바리'],
    ['Scorpionfish', '쏨뱅이']], dtype=object)

In [3]:
eng_kor = raw.values.tolist()

In [4]:
print(len(eng_kor))
print(eng_kor[:3])

57
[['Flounder', '문치가자미'], ['Sole', '돌가자미'], ['BlackSeaBream', '감성돔']]


In [5]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ']
# {c:i for i, c in enumerate(e_alpha)}
k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)
print('영어 알파벳 갯수 :', len(e_alpha))
print('한글 글자수 갯수 :', len(k_alpha))
# 4. 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐', 

In [6]:
# 4. 문자당 num를 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 47, 'E': 33, 'P': 44, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'Q': 45, 'R': 46, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, '가': 55, '각': 56, '간': 57, '감': 58, '개': 59, '거': 60, '것': 61, '게': 62, '계': 63, '고': 64, '관': 65, '광': 66, '구': 67, '굴': 68, '규': 69, '그': 70, '금': 71, '기': 72, '깊': 73, '나': 74, '날': 75, '남': 76, '내': 77, '넓': 78, '녀': 79, '노': 80, '놀': 81, '농': 82, '높': 83, '뉴': 84, '늦': 85, '다': 86, '단': 87, '도': 88, '동': 89, '들': 90, '람': 91, '랑': 92, '래': 93, '램': 94, '류': 95, '름': 96, '릎': 97, '리': 98, '많': 99, '망': 100, '매': 101, '머': 102, '먼': 103, '멍': 104, '메': 105, '명': 106, '모': 107, '목': 108, '무': 109, '물': 110, '미': 111, '바': 112, '반'

In [7]:
to_categorical([5, 3, 6], num_classes=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [8]:
def encoding(eng_kor=eng_kor):
    enc_in = []
    dec_in = []
    dec_out = []
    max_len_enc = max(len(data[0]) for data in eng_kor)
    max_len_dec = max(len(data[1]) for data in eng_kor) + 1  # 'S' 추가

    for data in eng_kor:
        # 인코딩 입력 데이터(영어->숫자->원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_padded = eng + [0] * (max_len_enc - len(eng))  # 패딩 추가
        eng_one = np.eye(alpha_total_size)[eng_padded]
        enc_in.append(eng_one)

        # 디코더 입력 데이터("S한글"->숫자->원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_padded = kor + [0] * (max_len_dec - len(kor))  # 패딩 추가
        kor_one = np.eye(alpha_total_size)[kor_padded]
        dec_in.append(kor_one)

        # 디코더 출력 데이터 ("한글E"->숫자)
        kor_out = [char_to_num[c] for c in data[1]+'E']
        kor_out_padded = kor_out + [0] * (max_len_dec - len(kor_out))  # 패딩 추가
        dec_out.append(kor_out_padded)

    return np.array(enc_in), np.array(dec_in), np.array(dec_out)


# 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding(eng_kor)

# 최종 입출력 데이터 shape 확인
print(X_enc.shape, X_dec.shape, Y_dec.shape)

(57, 23, 245) (57, 9, 245) (57, 9)


In [9]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(None, alpha_total_size))
_, state_h, state_c = LSTM(units=MY_HIDDEN, return_state=True)(ENC_IN)
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(None, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, return_sequences=True)(DEC_IN, initial_state=link)
DEC_OUT = Dense(units=alpha_total_size, activation='softmax')(DEC_MID)

# 모델 구조 구현
model = Model(inputs=[ENC_IN, DEC_IN], outputs=DEC_OUT)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None, 245)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None, 245)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 1024), (None,     │       5,201,920 │ input_layer[0][0]          │
│                               │ 1024), (None, 1024)]      │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ (None, None, 1024)        │       5,201,920 │ input_layer_1[0][0],       │
│                               │                           │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 245)         │         251,125 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,654,965 (40.65 MB)

 Trainable params: 10,654,965 (40.65 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
begin = time()
model.fit([X_enc, X_dec],
          Y_dec,
          epochs=1000,
          verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1687 - loss: 5.3859
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5710 - loss: 3.2840
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.2424 - loss: 4.7699
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5780 - loss: 1.9888
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5797 - loss: 1.7856
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5704 - loss: 1.8575
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.6065 - loss: 2.0297
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.6022 - loss: 1.6916
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5935 - loss: 1.6517
Epoch 10/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5918 - loss: 1.7856
Epoch 11/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6011 - loss: 1.7854
Epoch 12/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6352 - loss: 1.5707
E

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6570 - loss: 1.4080
Epoch 71/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6493 - loss: 1.4146
Epoch 72/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6634 - loss: 1.5772
Epoch 73/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6440 - loss: 1.3614
Epoch 74/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6556 - loss: 1.4748
Epoch 75/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6594 - loss: 1.4901
Epoch 76/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6651 - loss: 1.3326
Epoch 77/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6700 - loss: 1.3031
Epoch 78/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6495 - loss: 1.3750
Epoch 79/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6820 - loss: 1.2756
Epoch 80/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6486 - loss: 1.3439
Epoch 81/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6892 - loss: 1.2828
Epoch 

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7117 - loss: 1.0821
Epoch 140/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7233 - loss: 1.0092
Epoch 141/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7151 - loss: 1.0183
Epoch 142/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7086 - loss: 1.0723
Epoch 143/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6658 - loss: 1.6384
Epoch 144/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6950 - loss: 1.2064
Epoch 145/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7132 - loss: 1.0361
Epoch 146/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7094 - loss: 1.0183
Epoch 147/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7164 - loss: 1.1636
Epoch 148/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6745 - loss: 1.2881
Epoch 149/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6849 - loss: 1.1882
Epoch 150/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6785 - loss: 1.

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7672 - loss: 0.8120
Epoch 208/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7537 - loss: 0.7765
Epoch 209/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7346 - loss: 0.9043
Epoch 210/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7116 - loss: 1.2205
Epoch 211/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7425 - loss: 0.8045
Epoch 212/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7461 - loss: 0.7757
Epoch 213/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7524 - loss: 0.7838
Epoch 214/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7083 - loss: 0.9712
Epoch 215/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6936 - loss: 1.0536
Epoch 216/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7193 - loss: 1.0665
Epoch 217/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7425 - loss: 0.7760
Epoch 218/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7525 - loss: 0.

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7250 - loss: 1.1017
Epoch 276/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7363 - loss: 0.9270
Epoch 277/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7902 - loss: 0.6768
Epoch 278/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7883 - loss: 0.6421
Epoch 279/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7863 - loss: 0.6496
Epoch 280/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7684 - loss: 0.6761
Epoch 281/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7321 - loss: 0.8696
Epoch 282/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7191 - loss: 1.0517
Epoch 283/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7860 - loss: 0.7079
Epoch 284/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7759 - loss: 0.6362
Epoch 285/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8060 - loss: 0.6258
Epoch 286/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.7705 - loss: 0.

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8835 - loss: 0.3270
Epoch 684/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.8889 - loss: 0.2896
Epoch 685/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9273 - loss: 0.1776
Epoch 686/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9499 - loss: 0.1467
Epoch 687/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9458 - loss: 0.1337
Epoch 688/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9437 - loss: 0.1352
Epoch 689/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9483 - loss: 0.1315
Epoch 690/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 816ms/step - accuracy: 0.9546 - loss: 0.1347
Epoch 691/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9536 - loss: 0.1272   
Epoch 692/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9571 - loss: 0.1287
Epoch 693/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9474 - loss: 0.1316
Epoch 694/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9512 - lo

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9390 - loss: 0.1468
Epoch 752/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step - accuracy: 0.9474 - loss: 0.1155
Epoch 753/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step - accuracy: 0.9499 - loss: 0.1301
Epoch 754/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 446ms/step - accuracy: 0.9386 - loss: 0.1554
Epoch 755/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step - accuracy: 0.9324 - loss: 0.1417
Epoch 756/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 943ms/step - accuracy: 0.9512 - loss: 0.1208
Epoch 757/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9523 - loss: 0.1146
Epoch 758/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9461 - loss: 0.1231
Epoch 759/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9437 - loss: 0.1222
Epoch 760/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9510 - loss: 0.1071
Epoch 761/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9398 - loss: 0.1315
Epoch 762/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 750ms/step - accuracy:

Epoch 819/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9006 - loss: 0.3226
Epoch 820/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 900ms/step - accuracy: 0.9350 - loss: 0.1333
Epoch 821/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 710ms/step - accuracy: 0.9598 - loss: 0.0952
Epoch 822/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step - accuracy: 0.9611 - loss: 0.1012
Epoch 823/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step - accuracy: 0.9639 - loss: 0.1001
Epoch 824/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9686 - loss: 0.0843
Epoch 825/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9701 - loss: 0.0861
Epoch 826/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9662 - loss: 0.0802
Epoch 827/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9659 - loss: 0.0865
Epoch 828/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 688ms/step - accuracy: 0.9298 - loss: 0.2362
Epoch 829/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 902ms/step - accuracy: 0.8944 - loss: 0.4000
Epoch 830/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/s

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9724 - loss: 0.0678
Epoch 888/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9760 - loss: 0.0586
Epoch 889/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9763 - loss: 0.0615   
Epoch 890/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9738 - loss: 0.0619
Epoch 891/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9711 - loss: 0.0601
Epoch 892/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 858ms/step - accuracy: 0.9637 - loss: 0.0733
Epoch 893/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 945ms/step - accuracy: 0.9503 - loss: 0.1174
Epoch 894/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9701 - loss: 0.0676
Epoch 895/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9762 - loss: 0.0613
Epoch 896/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9711 - loss: 0.0715
Epoch 897/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9671 - loss: 0.0799
Epoch 898/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 942ms/step - accuracy: 0.961

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9688 - loss: 0.0542
Epoch 956/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9775 - loss: 0.0500
Epoch 957/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 702ms/step - accuracy: 0.9738 - loss: 0.0512
Epoch 958/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 685ms/step - accuracy: 0.9723 - loss: 0.0500
Epoch 959/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9699 - loss: 0.0738
Epoch 960/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9749 - loss: 0.0558
Epoch 961/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 726ms/step - accuracy: 0.9686 - loss: 0.0590
Epoch 962/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 607ms/step - accuracy: 0.9760 - loss: 0.0551
Epoch 963/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step - accuracy: 0.9749 - loss: 0.0573
Epoch 964/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 799ms/step - accuracy: 0.9701 - loss: 0.0666
Epoch 965/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 561ms/step - accuracy: 0.9619 - loss: 0.1142
Epoch 966/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step - acc

In [11]:
# 9. 모델 사용
test = [['Prawn', 'PP'],
        ['Icefish', 'PP'],
        ['Crab', 'PP'],
        ['SpottedSeaBass', 'PPP'],
        ['YellowPuffer', 'PP'],
        ['RockBream', 'PPPP'],
        ['WebfootOctopus', 'PPP']]
enc_in, dec_in, _ = encoding(test)

pred = model.predict([enc_in, dec_in])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step


In [12]:
for i in range(len(pred)):
    eng = test[i][0]
    hat = pred[i].argmax(axis=1)
    kor = alpha[hat[0]] + alpha[hat[1]]
    print("{} -> {}({})".format(eng, kor, hat))

Prawn -> 왕장([231 166  33   0   0])
Icefish -> 버들([226  90 140   0   0])
Crab -> 돌가([199  55  33   0   0])
SpottedSeaBass -> 황어([222 140  33  33   0])
YellowPuffer -> 점농([221  82 140   0   0])
RockBream -> 메어([105 140  33  33   0])
WebfootOctopus -> 참어([213 140  33  33   0])


In [14]:
model.save("model_translation.h5")